# CheXScan Ensemble Experiment

## Libraries

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Define transforms for validation or test data

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Define validation or test dataset

In [ ]:
data_dir = "path_to_your_data_directory"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

## Define dataloader

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

## Function to evaluate model accuracy

In [ ]:
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

## Load saved models

In [ ]:
model_vgg16 = load_saved_model('vgg16_model.pth')
model_densenet121 = load_saved_model('densenet121_model.pth')
model_resnet18 = load_saved_model('resnet18_model.pth')
model_inception_v3 = load_saved_model('inception_v3_model.pth')
model_alexnet = load_saved_model('alexnet_model.pth')
model_chexnet = load_saved_model('chexnet_model.pth')

## Evaluate accuracy of each model

In [ ]:
accuracy_vgg16 = evaluate_model(model_vgg16)
accuracy_densenet121 = evaluate_model(model_densenet121)
accuracy_resnet18 = evaluate_model(model_resnet18)
accuracy_inception_v3 = evaluate_model(model_inception_v3)
accuracy_alexnet = evaluate_model(model_alexnet)
accuracy_chexnet = evaluate_model(model_chexnet)

## Create a dictionary to store model accuracies

In [ ]:
model_accuracies = {
    'vgg16': accuracy_vgg16,
    'densenet121': accuracy_densenet121,
    'resnet18': accuracy_resnet18,
    'inception_v3': accuracy_inception_v3,
    'alexnet': accuracy_alexnet,
    'chexnet': accuracy_chexnet
}

## Get top 3 models based on accuracy

In [ ]:
top_models = sorted(model_accuracies.items(), key=lambda x: x[1], reverse=True)[:3]
print("Top 3 models based on accuracy:")
for model, accuracy in top_models:
    print(f"{model}: {accuracy}")

## Load saved models for the top 3 models

In [ ]:
top_model_names = [model_name for model_name, _ in top_models]
top_models = {}
for model_name in top_model_names:
    top_models[model_name] = load_saved_model(f"{model_name}_model.pth")

## Compute weights for the top 3 models based on their accuracies

In [ ]:

total_accuracy = sum(accuracy for _, accuracy in top_models.items())
weights = {model_name: accuracy / total_accuracy for model_name, accuracy in top_models.items()}

# Make predictions using the top 3 models
def predict_top_models(images):
    ensemble_prediction = torch.zeros(len(images), 3)
    for model_name, model in top_models.items():
        model.eval()
        outputs = model(images)
        ensemble_prediction += weights[model_name] * torch.softmax(outputs, dim=1)
    return ensemble_prediction

# Assuming you have a batch of images, you can use this function to get predictions
images = torch.randn(10, 3, 224, 224)  # Example batch of images
ensemble_predictions = predict_top_models(images)

# Convert predictions to class labels and confidence scores
class_names = ['Normal', 'Pneumonia', 'Tuberculosis']
predictions_with_confidence = []
for prediction in ensemble_predictions:
    confidence_scores = prediction.tolist()
    predicted_class_index = prediction.argmax().item()
    predicted_class = class_names[predicted_class_index]
    confidence = confidence_scores[predicted_class_index] * 100
    prediction_with_confidence = {predicted_class: confidence}
    predictions_with_confidence.append(prediction_with_confidence)

# Print predictions with confidence
for i, prediction in enumerate(predictions_with_confidence):
    print(f"Prediction for image {i + 1}: {prediction}")